In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from cmath import nan
import pywt
import torch
import pywt
from pandarallel import pandarallel
import tqdm
from functools import partial
from pathlib import Path
pandarallel.initialize(progress_bar=True)

/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
from numpy.lib.stride_tricks import as_strided

def sliding_window(data, win=29, s=1):
    itemsize  = data.itemsize
    N = len(data)
    sliding_data = as_strided(data, shape=((N - win) // s + 1, win), strides=(itemsize*s, itemsize))
    return sliding_data

In [3]:
def convert_canid_bits(cid):
    try:
        s = bin(int(str(cid), 16))[2:].zfill(29)
    except:
        s = bin(int(str(int(float(cid))), 16))[2:].zfill(29)
    bits = np.array(list(map(int, list(s))))
    return bits

In [4]:
label_to_num = {
    'Normal': 0,
    'Fuzzing': 1, 
    'Replay': 2
   }
def preprocess_per_row(sample):
   sample['bid']  = convert_canid_bits(sample['CAN_ID'])
   sample['label'] = label_to_num[sample['Label']]
   return sample

In [5]:
data_path = Path('../../../Data/LISA/Federated_Data/Labeled data/')
car_model = 'Tesla'
label = 'Normal'
file_id = 1
file_name = f'{car_model}_{label}_{file_id}.csv'
df = pd.read_csv(data_path / file_name)
df.CAN_ID = df.CAN_ID.astype(str)
# df = df.drop(['Time_Gap'], axis=1)
df

,Time_Ofset,Type,CAN_ID,Data_Length,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Time_Gap,Scaled_CAN_ID,Label
0,2.7,Rx,142,5,45,2,0,0,0,NaN,NaN,NaN,0.2,13,Normal
1,2.9,Rx,175,8,A0,A1,31,78,86,C6,0,AC,0.2,17,Normal
2,3.1,Rx,118,8,61,2,85,18,29,80,0,0,0.2,7,Normal
3,3.3,Rx,336,4,1D,1,B1,0,NaN,NaN,NaN,NaN,0.2,44,Normal
4,3.5,Rx,267,8,12,0,20,0,FA,12,7E,FF,0.2,31,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,334994.5,Rx,039D,5,26,5D,1,28,0,NaN,NaN,NaN,0.2,55,Normal
1048571,334995.4,Rx,101,8,15,0,42,0,14,40,25,D2,0.9,2,Normal
1048572,334995.6,Rx,111,8,77,FF,1C,1,EA,FF,27,B5,0.2,6,Normal
1048573,334995.9,Rx,175,8,56,C5,AA,60,95,AC,B2,8E,0.3,17,Normal


In [6]:
df = df.parallel_apply(preprocess_per_row, axis=1)

In [14]:
len_bid = df.bid.apply(lambda x: len(x)).to_numpy()
np.where(len_bid != 29)

(array([    225,     538,     849, ..., 1047922, 1048227, 1048543]),)

In [18]:
df.iloc[225]

Time_Ofset                                                    74.0
Type                                                            Rx
CAN_ID                                                    2.00E+08
Data_Length                                                      8
Data_0                                                           1
Data_1                                                           0
Data_2                                                           0
Data_3                                                           0
Data_4                                                           0
Data_5                                                           0
Data_6                                                          C0
Data_7                                                          AB
Time_Gap                                                       0.2
Scaled_CAN_ID                                                   38
Label                                                       No

In [25]:
df.CAN_ID.unique()

array(['142', '175', '118', '336', '267', '038C', '038D', '105', '011D',
       '129', '145', '155', '185', '1.00E+05', '187', '02D5', '186',
       '385', '038B', '101', '31', '51', '111', '370', '038E', '018A',
       '256', '108', '136', '206', '148', '257', '119', '011A', '240',
       '02A8', '222', '299', '221', '311', '038F', '488', '263', '027D',
       '286', '030A', '031A', '02B9', '039D', '03A2', '3.00E+01', '03A3',
       '03A4', '104', '2.00E+08', '211', '03A5', '03A6', '745', '04F6',
       '4.00E+01', '04F1', '331', '389', '032B', '039B', '421', '392',
       '03D1'], dtype=object)

In [27]:
df[df['CAN_ID']=='2.00E+08']

,Time_Ofset,Type,CAN_ID,Data_Length,Data_0,Data_1,Data_2,Data_3,Data_4,Data_5,Data_6,Data_7,Time_Gap,Scaled_CAN_ID,Label,bid,label
225,74.0,Rx,2.00E+08,8,1,0,0,0,0,0,C0,AB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
538,173.9,Rx,2.00E+08,8,1,0,0,0,0,0,D0,BB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
849,273.9,Rx,2.00E+08,8,1,0,0,0,0,0,E0,CB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1159,373.9,Rx,2.00E+08,8,1,0,0,0,0,0,F0,DB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1472,473.8,Rx,2.00E+08,8,1,0,0,0,0,0,0,EB,0.3,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047288,334586.3,Rx,2.00E+08,8,1,0,0,0,0,0,D0,BB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1047611,334687.6,Rx,2.00E+08,8,1,0,0,0,0,0,E0,CB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1047922,334787.6,Rx,2.00E+08,8,1,0,0,0,0,0,F0,DB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1048227,334886.3,Rx,2.00E+08,8,1,0,0,0,0,0,0,EB,0.2,38,Normal,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [24]:
bin(int(str(int(float(df.iloc[225].CAN_ID))), 16))

'0b1000000000000000000000000000000000'

In [8]:
window_size = 29 
strided = 13
can_id_sequences = sliding_window(df.bid.to_numpy(), win=window_size, s=strided)
can_id_sequences = np.array([np.stack(x) for x in can_id_sequences]).astype('int8')
label_sequences = sliding_window(df.label.to_numpy(), win=window_size, s=strided)
labels = np.any(label_sequences, axis=1).astype('int8')
labels[np.where(labels > 0)] = label_sequences[0][0]

ValueError: all input arrays must have the same shape

In [ ]:
output_path = Path('../../../Data/LISA/Federated_Data/Preprocessed_Data/')
output_path.mkdir(parents=True, exist_ok=True)
np.savez_compressed(output_path / file_name[:-4], X=can_id_sequences, y=labels)

## Load data

In [22]:
data_path = Path('../../../Data/LISA/Federated_Data/Preprocessed_Data/')
car_model = 'Kia'
label = 'Fuzz'
file_id = 1
file_name = f'{car_model}_{label}_{file_id}.npz'

In [23]:
data = np.load(data_path / file_name)

In [24]:
X, y = data['X'], data['y']
print(X.shape)
print(y.shape)

(35868, 29, 29)
(35868,)
